In [2]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from myfunc import text_clean

In [6]:
url = "https://www.yes24.com/product/category/bestseller"
payload = dict(categoryNumber="001", pageNumber=3, pageSize=24)
r = requests.get(url, params=payload)
soup = bs(r.content, "lxml")
soup

<!DOCTYPE html>
<html lang="ko">
<head><link href="https://m.yes24.com/Home/Best?DispNo=001" media="only screen and(max-width: 640px)" rel="alternate"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="dpr, width, viewport-width, rtt, downlink, ect, UA, UA-Platform, UA-Arch, UA-Model, UA-Mobile, UA-Full-Version" http-equiv="Accept-CH"/>
<meta content="86400" http-equiv="Accept-CH-Lifetime"/>
<meta content="unsafe-url" name="referrer"/>
<meta content="width=1170" name="viewport"/>
<title> 국내도서 종합 베스트 - 예스24 </title>
<meta content="예스24" name="title"/>
<meta content="도서, 티켓, 음반, 굿즈, 커뮤니티까지 문화 전반을 아우르는 국내 대표 문화콘텐츠 플랫폼, 아낌없는 혜택과 총알배송을 경험해보세요." name="description"/>
<meta content="인터넷 서점, 온라인 쇼핑, 상품 추천, 쇼핑몰, 상품 검색, 도서 정보, 국내도서, 외국도서, 전자책, eBook, 이북, 크레마, 공연, 콘서트, 뮤지컬, 음반, 예매, DVD, 블루레이, 예스24, YES24, 교보문고, 알라딘, 리센스, 예스24 도서용품, 친환경 PB 브랜드, 사은품, 굿즈" name="keywords"/>
<link href="//www.yes24.com/OpensearchDe

종류, 책제목, 상세페이지링크,  저자/역/글그림/그림, 출판사, 출간일, 가격, 회원리뷰수, 평점 

In [7]:
soup.select_one(".gd_name")["href"].split("/")[-1]

'125557465'

In [37]:
all_result = []
for item in soup.select("#yesBestList > li"):
    result = {}
    book_id = item.select_one(".gd_name")["href"].split("/")[-1]
    # 책종류
    kind = text_clean(item.select_one(".gd_res").get_text())
    # 책제목
    book_title = item.select_one(".gd_name").get_text()
    # 상세페이지링크 
    detail_link = "https://www.yes24.com" + item.select_one(".gd_name")["href"]
    # 저자/역자
    author = auth_extract(item.select_one(".authPub.info_auth").get_text())
    # 출판사
    publisher = item.select_one(".authPub.info_pub").get_text()
    # 출간일
    pub_date = item.select_one(".authPub.info_date").get_text()
    # 책가격
    book_price = int(item.select_one("div.info_row.info_price em.yes_b").get_text().replace(",", ""))
    # 사용자 리뷰수
    n_reviews = int(item.select_one(".txC_blue").get_text().replace(",", "")) if item.select_one(".txC_blue") != None else 0
    # 리뷰 평점
    rating = float(item.select_one(".rating_grade em.yes_b").get_text()) if item.select_one(".rating_grade em.yes_b") != None else 0
    result['book_id'] = book_id
    result['kind'] = kind
    result['book_title'] = book_title
    result['detail_link'] = detail_link
    result = result | author
    result['publisher'] = publisher
    result['pub_date'] = pub_date
    result['book_price'] = book_price
    result['n_reviews'] = n_reviews
    result['rating'] = rating
    all_result.append(result)

df = pd.DataFrame(all_result)
df

,book_id,kind,book_title,detail_link,저,publisher,pub_date,book_price,n_reviews,rating,원저,글,그림,감수,역,편저
0,125557465,도서,하루 한 장 나의 어휘력을 위한 필사 노트,https://www.yes24.com/product/goods/125557465,유선경,위즈덤하우스,2024년 10월,21420,884,9.5,NaN,NaN,NaN,NaN,NaN,NaN
1,167627172,도서,흔한남매 21,https://www.yes24.com/product/goods/167627172,NaN,미래엔아이세움,2025년 12월,15120,104,9.7,흔한남매,백난도,유난희,흔한컴퍼니,NaN,NaN
2,171432215,도서,뇌는 어떻게 나를 조종하는가,https://www.yes24.com/product/goods/171432215,크리스 나이바우어,클랩북스,2026년 01월,16920,30,9.6,NaN,NaN,NaN,NaN,김윤종,NaN
3,169629856,도서,쓰면서 바로 써먹는 초등 어휘 따라 쓰기,https://www.yes24.com/product/goods/169629856,NaN,피넛,2026년 01월,15120,25,10.0,NaN,올바른초등교육연구소,고도연,NaN,NaN,NaN
4,142687238,도서,어른의 품격을 채우는 100일 필사 노트,https://www.yes24.com/product/goods/142687238,김종원,청림Life,2025년 03월,18000,249,9.8,NaN,NaN,NaN,NaN,NaN,NaN
5,169689075,도서,내가 선택할 수 있는 품격 있는 태도에 관하여,https://www.yes24.com/product/goods/169689075,김종원,오아시스,2025년 12월,17100,75,9.9,NaN,NaN,NaN,NaN,NaN,NaN
6,171015710,도서,2026 심우철 실전 동형 모의고사 Season 1,https://www.yes24.com/product/goods/171015710,심우철,에스티유니타스,2025년 12월,13500,49,10.0,NaN,NaN,NaN,NaN,NaN,NaN
7,174667883,도서,떠난 것은 돌아오지 않는다,https://www.yes24.com/product/goods/174667883,줄리언 반스,다산북스,2026년 01월,16200,19,9.7,NaN,NaN,NaN,NaN,정영목,NaN
8,171520765,도서,2026 선재국어 결승선 봉투 모의고사,https://www.yes24.com/product/goods/171520765,NaN,수비니겨,2026년 01월,25200,29,10.0,NaN,NaN,NaN,NaN,NaN,이선재
9,153064968,도서,트렌드 코리아 2026,https://www.yes24.com/product/goods/153064968,"[김난도, 전미영, 최지혜, 권정윤, 한다혜, 이혜원, 이수진, 서유현, 전다현, ...",미래의창,2025년 09월,18000,243,9.2,NaN,NaN,NaN,NaN,NaN,NaN


# 저자 분리하기

In [11]:
soup.select("#yesBestList > li")

[<li class="" data-goods-no="125557465" data-iy-no="0" data-statgb="02">
 <div class="itemUnit">
 <div class="item_img">
 <div class="img_canvas">
 <div class="img_upper">
 <em class="ico rank">49</em>
 <span class="rank_info rank_up">
 <span class="ico"></span><em class="txt rank">1</em><em class="txt">위 상승</em>
 </span>
 </div>
 <span class="img_item">
 <span class="img_grp">
 <a class="lnk_img" href="/product/goods/125557465" onclick="wiseLogV2('BS', '001_005_001', ''); ">
 <em class="img_bdr">
 <img alt="하루 한 장 나의 어휘력을 위한 필사 노트" border="0" class="lazy" data-original="https://image.yes24.com/goods/125557465/L" src="https://image.yes24.com/momo/Noimg_L.jpg"/>
 </em>
 </a>
 </span>
 </span>
 </div>
 <div class="img_btn">
 <a class="btnC btn_preview" href="javascript:yes24GU.openPreviewCheck(125557465); wiseLogV2('BS', '001_005_011', '');"><span class="bWrap"><em class="txt">미리보기</em></span></a>
 </div>
 </div>
 <div class="item_info">
 <div class="info_row info_keynote">
 <span class=

In [16]:
def auth_extract(authors):
    result = {} # 1권의 저자 정보가 모임
    if "보기/감추기" in authors:
        temp = authors.split("보기/감추기")
        author = temp[1].strip().split("\n")
        result['저'] = author
    #         print(author)
    else:
        for name_by_role in item.select_one(".authPub.info_auth").get_text().split("/"):
            temp = name_by_role.split()
            name = " ".join(temp[:-1])
            role = temp[-1]
            result[role] = name

    return result

In [25]:
for item in soup.select("#yesBestList > li"):
    author = auth_extract(item.select_one(".authPub.info_auth").get_text())
    print(author)

{'저': '유선경'}
{'원저': '흔한남매', '글': '백난도', '그림': '유난희', '감수': '흔한컴퍼니'}
{'저': '크리스 나이바우어', '역': '김윤종'}
{'글': '올바른초등교육연구소', '그림': '고도연'}
{'저': '김종원'}
{'저': '김종원'}
{'저': '심우철'}
{'저': '줄리언 반스', '역': '정영목'}
{'편저': '이선재'}
{'저': ['김난도', '전미영', '최지혜', '권정윤', '한다혜', '이혜원', '이수진', '서유현', '전다현', '이준영', '이향은', '김나은']}
{'저': '다크 사이드 프로젝트'}
{'저': '김미영'}
{'저': '칼 에드워드 세이건', '역': '홍승수'}
{'저': ['이반 투르게네프', '토마스 하디', '기 드 모파상', '셔우드 앤더슨', '안톤 파블로비치 체호프', '허먼 멜빌', '에드가 앨런 포', '레프 톨스토이', '마이너스']}
{'저': '양승아'}
{'저': 'ETS'}
{'저': '마이클 이스터', '역': '김원진'}
{'저': '무라카미 하루키', '역': '임홍빈'}
{'저': '한강'}
{'글': '김수미', '그림': '유루시아'}
{'저': '윤서진'}
{'저': '최영원'}
{'저': '손자', '역': '소준섭'}
{'저': 'ETS'}


In [34]:
tuple(author.keys())

('저',)

In [23]:
author

'ETS 저                            '

In [27]:
{result2 | author }

TypeError: unhashable type: 'dict'

# yes24 종합베스트 999권 자료 수집해서 csv 파일로 저장하기 + 저자 분리저장
* book_data/
* yes24_수집일_best.csv
* 수집일은 datatime에서 오늘 날짜 추출해서 파일명에 포함시키기

In [39]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from myfunc import text_clean

In [40]:
from datetime import datetime

In [41]:
today = datetime.today()
print(today)

2026-01-27 14:00:22.464639


In [42]:
today = f"{today.year}{today.month:02d}{today.day:02d}"
today

'20260127'

In [48]:
all_result = []
for page in range(1, 10):
    
    url = "https://www.yes24.com/product/category/bestseller"
    payload = dict(categoryNumber="001", pageNumber=page, pageSize=120)
    r = requests.get(url, params=payload)
#     print(r.status_code)
    soup = bs(r.content, "lxml")

    for idx, item in enumerate(soup.select("#yesBestList > li")):
        print(f"{page}/{9}page, {idx}/{len(soup.select('#yesBestList > li'))-1}   수집중", end="\r")
        result = {}
        book_id = item.select_one(".gd_name")["href"].split("/")[-1]
        # 책종류
        kind = text_clean(item.select_one(".gd_res").get_text())
        # 책제목
        book_title = item.select_one(".gd_name").get_text()
        # 상세페이지링크 
        detail_link = "https://www.yes24.com" + item.select_one(".gd_name")["href"]
        # 저자/역자
        author = auth_extract(item.select_one(".authPub.info_auth").get_text())
        # 출판사
        publisher = item.select_one(".authPub.info_pub").get_text()
        # 출간일
        pub_date = item.select_one(".authPub.info_date").get_text()
        # 책가격
        book_price = int(item.select_one("div.info_row.info_price em.yes_b").get_text().replace(",", ""))
        # 사용자 리뷰수
        n_reviews = int(item.select_one(".txC_blue").get_text().replace(",", "")) if item.select_one(".txC_blue") != None else 0
        # 리뷰 평점
        rating = float(item.select_one(".rating_grade em.yes_b").get_text()) if item.select_one(".rating_grade em.yes_b") != None else 0
        result['book_id'] = book_id
        result['kind'] = kind
        result['book_title'] = book_title
        result['detail_link'] = detail_link
        result = result | author
        result['publisher'] = publisher
        result['pub_date'] = pub_date
        result['book_price'] = book_price
        result['n_reviews'] = n_reviews
        result['rating'] = rating
        all_result.append(result)
            
    time.sleep(3)

dir_name = "book_data"
if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        print(f"{dir_name}을 생성했습니다.")        
        
df = pd.DataFrame(all_result)
df.to_csv(f"./{dir_name}/yes24_{today}_best_저자분리.csv")

In [43]:
df.to_csv(f"./{dir_name}/yes24_{today}_best.csv")

In [49]:
df

,book_id,kind,book_title,detail_link,저,역,publisher,pub_date,book_price,n_reviews,...,편저,원저,편,기획,등저,편역,공저,해설,사진,평역
0,172506508,도서,돈의 방정식,https://www.yes24.com/product/goods/172506508,모건 하우절,박영준,서삼독,2026년 01월,25200,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,164510819,도서,괴테는 모든 것을 말했다,https://www.yes24.com/product/goods/164510819,스즈키 유이,이지수,리프,2025년 11월,15300,195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,168226997,도서,진보를 위한 주식투자,https://www.yes24.com/product/goods/168226997,이광수,NaN,21세기북스,2025년 12월,19800,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148032819,도서,박곰희 연금 부자 수업,https://www.yes24.com/product/goods/148032819,박곰희,NaN,인플루엔셜,2025년 06월,18900,295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,167503385,도서,자본주의 시대에서 살아남기 위한 최소한의 경제 공부,https://www.yes24.com/product/goods/167503385,백억남(김욱현),NaN,하이스트,2025년 12월,25200,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,124923518,만화,장송의 프리렌 11,https://www.yes24.com/product/goods/124923518,NaN,NaN,학산문화사,2024년 02월,5400,149,...,NaN,야마다 카네히토,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,122865851,도서,이은경쌤의 사자성어 속담 일력 365,https://www.yes24.com/product/goods/122865851,이은경,NaN,포레스트북스,2023년 10월,19800,213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,122532297,도서,"돈, 뜨겁게 사랑하고 차갑게 다루어라",https://www.yes24.com/product/goods/122532297,앙드레 코스톨라니,한윤진,미래의창,2023년 09월,13500,118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,116745219,도서,베싸육아,https://www.yes24.com/product/goods/116745219,박정은,NaN,래디시,2023년 01월,19800,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# yes24_수집일_best.csv 파일에서 상세링크만 가져와서 상세페이지에 방문하고 
* book_id
* 쪽수, 무게, 크기
* 카테고리 분류 
* yes24_수집일_best_detail.csv

# 수집하기

In [ ]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from myfunc import text_clean

In [74]:
from datetime import datetime

In [81]:
today = datetime.today()
print(today)

2026-01-27 10:21:43.522528


In [86]:
today = f"{today.year}{today.month:02d}{today.day:02d}"
today

'20260127'

In [93]:
data = pd.read_csv("./book_data/yes24_20260127_best.csv")
detail_links = data['detail_link']
detail_links

0      https://www.yes24.com/product/goods/172506508
1      https://www.yes24.com/product/goods/164510819
2      https://www.yes24.com/product/goods/168226997
3      https://www.yes24.com/product/goods/148032819
4      https://www.yes24.com/product/goods/167503385
                           ...                      
994    https://www.yes24.com/product/goods/124923518
995    https://www.yes24.com/product/goods/122865851
996    https://www.yes24.com/product/goods/122532297
997    https://www.yes24.com/product/goods/116745219
998    https://www.yes24.com/product/goods/116599481
Name: detail_link, Length: 999, dtype: object

In [158]:
all_detail_info = []
for idx, url in enumerate(detail_links):
    print(f"{idx}/{len(detail_links)}   수집중", end="\r")
    r = requests.get(url)
    soup = bs(r.content, "lxml")
    
    book_info ={}
    
    # book_id
    book_id = url.split("/")[-1]
    book_info['book_id'] = book_id 
    
    # 쪽수, 무게, 크기
    for row in soup.select("tbody.b_size tr"):
        if "쪽수" in row.select_one("th").get_text():
            pummoc_info = row.select_one("td").get_text().split("|")
    
    print(pummoc_info)
    for item in pummoc_info:
        temp = item.strip()
        if temp[-1] == "쪽":
            book_info["페이지"] = temp
        elif temp[-1] == "g":
            book_info['무게'] = temp
        elif temp[-1] == "m":
            book_info['크기'] = temp

    #카테고리 분류
    category = soup.select_one("#infoset_goodsCate .yesAlertLi > li").get_text(strip=True).split(">")    
    book_info['카테고리'] = category
   
    all_detail_info.append(book_info)
    
#     time.sleep(3)

df2 = pd.DataFrame(all_detail_info)
df2

['372쪽 ', ' 584g ', ' 140*210*27mm']
['248쪽 ', ' 336g ', ' 128*188*17mm']
['260쪽 ', ' 408g ', ' 152*225*16mm']
['296쪽 ', ' 524g ', ' 152*225*20mm']
['604쪽 ', ' 856g ', ' 152*225*29mm']
['33쪽 ', ' 270g ', ' 194*237*9mm']
['352쪽 ', ' 528g ', ' 146*210*23mm']
['392쪽 ', ' 235*310*30mm']
['272쪽 ', ' 420g ', ' 135*210*18mm']
['544쪽 ', ' 1,368g ', ' 210*280*24mm']


,book_id,페이지,무게,크기,카테고리
0,172506508,372쪽,584g,140*210*27mm,"[국내도서, 경제 경영, 투자/재테크, 재무/자산관리, 재테크일반]"
1,164510819,248쪽,336g,128*188*17mm,"[국내도서, 소설/시/희곡, 일본소설, 일본 장편소설]"
2,168226997,260쪽,408g,152*225*16mm,"[국내도서, 경제 경영, 투자/재테크, 주식/ETF, 국내주식]"
3,148032819,296쪽,524g,152*225*20mm,"[국내도서, 경제 경영, 투자/재테크, 재무/자산관리, 연금]"
4,167503385,604쪽,856g,152*225*29mm,"[국내도서, 경제 경영, 경제/경제학, 경제상식/경제이야기]"
5,27582352,33쪽,270g,194*237*9mm,"[국내도서, 유아, 4-6세, 4-6세 다른나라 그림책]"
6,166390503,352쪽,528g,146*210*23mm,"[국내도서, 인문, 인문/교양, 교양으로 읽는 인문]"
7,168393009,392쪽,NaN,235*310*30mm,"[국내도서, 수험서 자격증, 한국사능력검정시험, 심화(1,2,3급)]"
8,147276346,272쪽,420g,135*210*18mm,"[국내도서, 에세이, 한국 에세이]"
9,168776448,544쪽,"1,368g",210*280*24mm,"[국내도서, 국어 외국어 사전, 영어, 토익/TOEIC, 독해/문법/RC]"


In [160]:
df2 = pd.DataFrame(all_detail_info)

dir_name = "book_data"
if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        print(f"{dir_name}을 생성했습니다.")        
        

df2.to_csv(f"./{dir_name}/yes24_{today}_best_detail.csv")
df2

,book_id,페이지,무게,크기,카테고리
0,172506508,372쪽,584g,140*210*27mm,"[국내도서, 경제 경영, 투자/재테크, 재무/자산관리, 재테크일반]"
1,164510819,248쪽,336g,128*188*17mm,"[국내도서, 소설/시/희곡, 일본소설, 일본 장편소설]"
2,168226997,260쪽,408g,152*225*16mm,"[국내도서, 경제 경영, 투자/재테크, 주식/ETF, 국내주식]"
3,148032819,296쪽,524g,152*225*20mm,"[국내도서, 경제 경영, 투자/재테크, 재무/자산관리, 연금]"
4,167503385,604쪽,856g,152*225*29mm,"[국내도서, 경제 경영, 경제/경제학, 경제상식/경제이야기]"
5,27582352,33쪽,270g,194*237*9mm,"[국내도서, 유아, 4-6세, 4-6세 다른나라 그림책]"
6,166390503,352쪽,528g,146*210*23mm,"[국내도서, 인문, 인문/교양, 교양으로 읽는 인문]"
7,168393009,392쪽,NaN,235*310*30mm,"[국내도서, 수험서 자격증, 한국사능력검정시험, 심화(1,2,3급)]"
8,147276346,272쪽,420g,135*210*18mm,"[국내도서, 에세이, 한국 에세이]"
9,168776448,544쪽,"1,368g",210*280*24mm,"[국내도서, 국어 외국어 사전, 영어, 토익/TOEIC, 독해/문법/RC]"


# 두개의 수집코드 합쳐서 한번에 모두 수집하기

In [1]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from myfunc import text_clean
from datetime import datetime

In [2]:
def today():
    today = datetime.today()
    today = f"{today.year}{today.month:02d}{today.day:02d}"
    return today

In [3]:
def auth_extract(authors):
    result = {} # 1권의 저자 정보가 모임
    if "보기/감추기" in authors:
        temp = authors.split("보기/감추기")
        author = temp[1].strip().split("\n")
        result['저'] = author
    else:
        for name_by_role in item.select_one(".authPub.info_auth").get_text().split("/"):
            temp = name_by_role.split()
            name = " ".join(temp[:-1])
            role = temp[-1]
            result[role] = name

    return result

In [8]:
def detail_info_extract(url):
    r = requests.get(url)
    soup2 = bs(r.content, "lxml")
    
    book_info ={}
   
    # 쪽수, 무게, 크기
    for row in soup2.select("tbody.b_size tr"):
        if "쪽수" in row.select_one("th").get_text():
            pummoc_info = row.select_one("td").get_text().split("|")
    
            for item in pummoc_info:
                temp = item.strip()
                if temp[-1] == "쪽":
                    book_info["페이지"] = temp
                elif temp[-1] == "g":
                    book_info['무게'] = temp
                elif temp[-1] == "m":
                    book_info['크기'] = temp

    #카테고리 분류
    try:
        category = soup2.select_one("#infoset_goodsCate .yesAlertLi > li").get_text(strip=True).split(">")    
    except:
        category = ""
    book_info['카테고리'] = category
    
    return book_info

In [7]:
all_result = []
for page in range(1, 10):
    
    url = "https://www.yes24.com/product/category/bestseller"
    payload = dict(categoryNumber="001", pageNumber=page, pageSize=120)
    r = requests.get(url, params=payload)
#     print(r.status_code)
    soup = bs(r.content, "lxml")

    for idx, item in enumerate(soup.select("#yesBestList > li")):
        print(f"{page}/{9}page, {idx}/{len(soup.select('#yesBestList > li'))-1}       수집중", end="\r")
        result = {}
        book_id = item.select_one(".gd_name")["href"].split("/")[-1]
        # 책종류
        kind = text_clean(item.select_one(".gd_res").get_text())
        # 책제목
        book_title = item.select_one(".gd_name").get_text()
        # 상세페이지링크 
        detail_link = "https://www.yes24.com" + item.select_one(".gd_name")["href"]
        # 저자/역자
        author = auth_extract(item.select_one(".authPub.info_auth").get_text())
        # 출판사
        publisher = item.select_one(".authPub.info_pub").get_text()
        # 출간일
        pub_date = item.select_one(".authPub.info_date").get_text()
        # 책가격
        book_price = int(item.select_one("div.info_row.info_price em.yes_b").get_text().replace(",", ""))
        # 사용자 리뷰수
        n_reviews = int(item.select_one(".txC_blue").get_text().replace(",", "")) if item.select_one(".txC_blue") != None else 0
        # 리뷰 평점
        rating = float(item.select_one(".rating_grade em.yes_b").get_text()) if item.select_one(".rating_grade em.yes_b") != None else 0
        
        detail_infos = detail_info_extract(detail_link)
        
        result['book_id'] = book_id
        result['kind'] = kind
        result['book_title'] = book_title
        result['detail_link'] = detail_link
        result = result | author
        result['publisher'] = publisher
        result['pub_date'] = pub_date
        result['book_price'] = book_price
        result['n_reviews'] = n_reviews
        result['rating'] = rating
        result = result | detail_infos
#         print(result)
        all_result.append(result)
            
#         time.sleep(2)

dir_name = "book_data"
if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        print(f"{dir_name}을 생성했습니다.")        
        
df = pd.DataFrame(all_result)
df.to_csv(f"./{dir_name}/yes24_{today()}_best_저자분리.csv")
print("데이터 수집 완료")

데이터 수집 완료38/38       수집중집중


In [6]:
detail_info_extract(detail_link)

{'페이지': '316쪽', '무게': '496g', '크기': '140*205*18mm', '카테고리': ''}